In [290]:
import pandas as pd
import re
from typing import List, Tuple, Dict, Set
import logging
from collections import defaultdict
import random

from main import *
logging.basicConfig(level=logging.DEBUG, format='[%(asctime)s] %(levelname)s: %(message)s')

In [2]:
# input_path  = r"G:\PROJ\220927\戎圩填字表用_方音調搽字表_220928.xlsx"
# locale_name = "戎墟"
# SHEET_NAME  = "主表"
# CHAR_COL    = 'H'
# PRON_COL    = "TUV"
# PRON_ND_COL = "WXY"
# MEAN_COL    = 'Z'

In [2]:
input_path  = r"C:\Users\52643\Desktop\端州.xlsx"
locale_name = "戎墟"
SHEET_NAME  = "審音"
CHAR_COL    = 'F'
PRON_COL    = "I"
PRON_ND_COL = ""
MEAN_COL    = 'J'

In [3]:
use_col = [ord(CHAR_COL) - 65, ord(MEAN_COL) - 65]
use_col.extend([ord(i) - 65 for i in PRON_COL])
use_col.extend([ord(i) - 65 for i in PRON_ND_COL] if PRON_ND_COL!=None else [])

locale_rule = [0, 1, locale_name]
get_ipa_for_locale = lambda pron: get_ipa(locale_rule, pron, locale_name)

data_file = pd.read_excel(input_path, sheet_name=SHEET_NAME, usecols=use_col, keep_default_na=False)

In [43]:
entry_list, chara_index_dict = read_sheet(data_file, len(PRON_COL), len(PRON_ND_COL), False)
for entry in entry_list: entry.rm_duplicate()
entry_list, chara_index_dict = sim_2_trad(entry_list, chara_index_dict, False)
for entry in entry_list: entry.toIpa(locale_rule, locale_name)

[2022-10-02 16:56:12,395] INFO: 總共 4041 行
[2022-10-02 16:56:12,673] INFO: 讀取 3550 行
[2022-10-02 16:56:12,675] INFO: 6 合併: 馱: [0]['to4'], [1]['to4'] => ['to4']
[2022-10-02 16:56:12,675] INFO: 8 合併: 大: [0]['daai6'], [1]['daai6'] => ['daai6']
[2022-10-02 16:56:12,676] INFO: 19 合併: 做: [0]['zou3'], [1]['zou3'] => ['zou3']
[2022-10-02 16:56:12,676] INFO: 44 合併: 簸: [0]['bo3']<顛～>, [1]['bo3']<～箕> => ['bo3']
[2022-10-02 16:56:12,676] INFO: 52 合併: 薄: [0]['bok6']<～荷>, [1]['bok6'] => ['bok6']<～荷>
[2022-10-02 16:56:12,677] INFO: 86 合併: 過: [0]['gwo3'], [1]['gwo3'] => ['gwo3']
[2022-10-02 16:56:12,677] INFO: 144 合併: 炸: [0]['zaa3'], [1]['zaa3'] => ['zaa3']
[2022-10-02 16:56:12,678] INFO: 154 合併: 灑: [0]['saa2'], [1]['saa2'] => ['saa2']
[2022-10-02 16:56:12,678] INFO: 177 合併: 嚇: [0]['haak3'], [1]['haak3']<恐～> => ['haak3']<恐～>
[2022-10-02 16:56:12,679] INFO: 183 合併: 夏: [0]['haa6']<姓>, [1]['haa6']<～天> => ['haa6']
[2022-10-02 16:56:12,679] INFO: 198 合併: 藉: [0]['zik6'], [1]['zik6'] => ['zik6']
[2022-10-02 1

In [44]:
print(entry_list[1010])
print(entry_list[1010].multiprons[0].split_prons[0])

止 => ['zi2']->['ʦi35']
['z', 'i', '', '2']


In [45]:
csv_file_path = 'data/KwangUon.csv'
df = pd.read_csv(csv_file_path,
                 usecols=["最簡描述", "字頭覈校前", "字頭"],
                 keep_default_na=False)
df.head()

,最簡描述,字頭覈校前,字頭
0,端一東平,東,
1,端一東平,菄,
2,端一東平,鶇,
3,端一東平,䍶,
4,端一東平,𠍀,


In [7]:
所有母 = '幫滂並明端透定泥來知徹澄孃精清從心邪莊初崇生俟章昌常書船日見溪羣疑影曉匣云以'
所有等 = '一二三四'
所有韻 = '東冬鍾江支脂之微魚虞模齊祭泰佳皆夬灰咍廢眞臻文欣元魂痕寒刪山仙先蕭宵肴豪歌麻陽唐庚耕清青蒸登尤侯幽侵覃談鹽添咸銜嚴凡'
所有聲 = '平上去入'
韻順序表 = '東_冬鍾江支脂之微魚虞模齊祭泰佳皆夬灰咍廢眞臻文欣元魂痕寒刪山仙先蕭宵肴豪歌_麻_陽唐庚_耕清青蒸登尤侯幽侵覃談鹽添咸銜嚴凡'
脣音母 = set('幫滂並明')
開合中立韻 = set('東冬鍾江虞模尤幽')
重紐母 = set('幫滂並明見溪羣疑影曉匣')
v1重紐韻 = set('支脂祭眞仙宵侵鹽清')
呼到韻 = {
    '開': set('咍痕欣嚴之魚臻蕭宵肴豪侯侵覃談鹽添咸銜'),
    '合': set('灰魂文凡'),
}
等到韻 = {
    '一': set('冬模泰咍灰痕魂寒豪唐登侯覃談'),
    '二': set('江佳皆夬刪山肴耕咸銜'),
    '三': set('鍾支脂之微魚虞祭廢眞臻欣元文仙宵陽清蒸尤幽侵鹽嚴凡'),
    '四': set('齊先蕭青添'),
}
攝到韻 = {
    "通":set("東冬鍾"),
    "江":set("江"),
    "止":set("支脂之微"),
    "遇":set("魚虞模"),
    "蟹":set("齊佳皆灰咍祭泰夬廢"),
    "臻":set("眞諄臻文欣元魂痕"),
    "山":set("寒桓刪山先仙"),
    "效":set("蕭宵肴豪"),
    "果":set("歌戈"),
    "假":set("麻"),
    "宕":set("唐陽"),
    "梗":set("庚耕清青"),
    "曾":set("登蒸"),
    "流":set("侯尤幽"),
    "深":set("侵"),
    "咸":set("覃談鹽添咸銜嚴凡")}
組到聲 = {
    "幫":set("幫滂並明"),
    "端":set("端透定泥"),
    "知":set("知徹澄孃"),
    "來":set("來"),
    "精":set("精清從心邪"),
    "莊":set("莊初崇生俟"),
    "章":set("章昌常書船"),
    "日":set("日"),
    "見":set("見溪羣疑"),
    "影":set("影云曉匣"),
    "以":set("以")}
PATTERN_描述 = re.compile(f'([{所有母}])([開合])?([{所有等}])?([AB])?([{所有韻}])([{所有聲}])')
取得地位 = lambda 描述: PATTERN_描述.fullmatch(描述).groups()

In [147]:
韻到等 = defaultdict()
for 等, 韻s  in 等到韻.items():
    for 韻 in 韻s: 韻到等[韻] = 等
韻到呼 = defaultdict()
for 呼, 韻s  in 呼到韻.items():
    for 韻 in 韻s: 韻到呼[韻] = 呼
for 韻 in 開合中立韻:
    韻到呼[韻] = "中"
韻到攝 = defaultdict()
for 攝, 韻s  in 攝到韻.items():
    for 韻 in 韻s: 韻到攝[韻] = 攝
聲到組 = defaultdict()
for 組, 聲s  in 組到聲.items():
    for 聲 in 聲s: 聲到組[聲] = 組

In [151]:
class Loc():
    聲: str = ""
    組: str = ""
    呼: str = ""
    等: str = ""
    紐: str = ""
    攝: str = ""
    韻: str = ""
    調: str = ""
    def __init__(self, loc: Tuple):
        self.聲, self.呼, self.等, self.紐, self.韻, self.調 = [l if l!=None else "" for l in loc]
        self.攝 = 韻到攝[self.韻]
        self.組 = 聲到組[self.聲]
        if self.等=="": self.等 = 韻到等[self.韻]
        if self.呼=="" and self.韻 in 韻到呼: self.呼 = 韻到呼[self.韻]
    def __str__(self):
        # return "聲: %s, 呼: %s, 等: %s, 紐: %s, 韻: %s, 調: %s, 攝: %s" % (self.聲, self.呼, self.等, self.紐, self.韻, self.調, self.攝)
        return f"{self.組}組{self.聲}母{self.攝}攝{self.等}等{self.韻}{self.紐}韻{self.調}聲"
    def get(self, name: str) -> str:
        if name=='聲': return self.聲
        elif name=='呼': return self.呼
        elif name=='等': return self.等
        elif name=='紐': return self.紐
        elif name=='韻': return self.韻
        elif name=='調': return self.調
        elif name=='攝': return self.攝
        elif name=='組': return self.組
        else: raise Exception("不支援的地位名稱: " + name)

In [10]:
def get_jpp_final(jpp_splited: List[str], rewrite_final=True):
    nuclei, final = jpp_splited[1:3]
    f = final
    if rewrite_final:
        if f=="p": f="m"
        if f=="t": f="n"
        if f=="k": f="ng"
    return nuclei + f

In [152]:
chara_to_locate: Dict[str, List[Loc]] = defaultdict(list)
for i in df.itertuples():
    if not i.最簡描述: continue
    chara_head = i.字頭 if i.字頭!="" else i.字頭覈校前
    chara_to_locate[chara_head].append(Loc(取得地位(i.最簡描述)))
print(chara_to_locate['重'][0])

知組澄母通攝三等鍾韻平聲


In [12]:
# for finals  : 紐<-呼<-韻<-攝<-聲<-調
# for initials: 

In [441]:
class PronNode():
    children: Dict[str, 'PronNode'] # name -> PronNode
    names   : str # names of the Loc # 本節點的地位
    type    : str # 'final' or 'initial' or what # 本節點的類型
    charas  : str # all charas that belongs to the Loc # 音韻地位下的轄字
    prons   : Dict[str, set] # pron of the charas # 時音-轄字
    def __init__(self, name: str, type: str) -> None:
        self.names    = name
        self.type     = type
        self.charas   = ""
        self.children = defaultdict(PronNode)
        self.prons    = defaultdict(set)
    def __repr__(self) -> str:
        return self.repr();
    def repr(self, show_charas=False):
        this_layer = self.names + "[" + self.type +"]: " + (self.charas if show_charas else "")
        if len(self.children)>0: this_layer += " { " + "".join([v.names for k, v in self.children.items()]) + " } "
        if len(self.prons)>0: this_layer += ("\n=時音: " if show_charas else "") + ", ".join(["%s: %s" % (k,"".join(v)) for k, v in self.prons.items()])
        return this_layer
    def __str__(self) -> str:
        return self.str()
    def str(self, indent_mark:str=" ", layer:int=0, show_charas=True) -> str:
        this_layer = (indent_mark*layer) + "-" + self.names + "[" + self.type +"]: " + (self.charas if show_charas else "")
        if len(self.prons)>0: this_layer += (("\n"+(indent_mark*(layer+1))+"=時音: ") if show_charas else "") + ", ".join(["%s: %s" % (k,"".join(v)) for k, v in self.prons.items()])
        if len(self.children) == 0: return this_layer
        next_layer = "\n".join(child.str(indent_mark, layer+1, show_charas) for child in self.children.values())
        return this_layer + "\n" + next_layer
    def fold(self, thre=0.8, fold_empty=False) -> None:
        id = random.randint(0, 1000000)
        print("FOLDING %07d: %s" % (id, self.__repr__()))
        if len(self.children) == 0: 
            pass # this is a leaf node
        elif len(self.children) == 1: # this node has only one child
            child_0 = self.children[list(self.children)[0]]
            child_0.fold(thre=thre, fold_empty=fold_empty)
            # if child_0.isleaf():
            self.charas = child_0.charas
            self.prons  = child_0.prons
            self.children = child_0.children
        elif len(self.children)>1:
            prons:Dict[str, set] = defaultdict(set)
            prons_chs = ""
            charas = ""
            need_fold = []
            is_empty = []
            for child in self.children.values():
                if not child.isleaf(): child.fold(thre=thre, fold_empty=fold_empty)
                if child.isleaf():
                    for pron, pron_chs in child.prons.items():
                        prons[pron] = prons[pron].union(pron_chs)
                        prons_chs += "".join(pron_chs)
                        charas += child.charas
                    need_fold.append(True)
                    is_empty.append(len(child.prons)==0)
                else:
                    need_fold.append(False)
                    is_empty.append(False)
                print("children: ", child.__repr__(), ", need_fold: ", need_fold[-1])
            print(f"need_fold: {need_fold}, is_empty: {is_empty}")
            if all(is_empty):
                self.charas = charas
                self.children = defaultdict(PronNode)
                self.prons = prons
                print("FOLD because of empty node")
            else:
                prons_percent = []
                for pron, pron_chs in prons.items():
                    prons_percent.append((len(pron_chs)+3)/(len(prons_chs)+3))
                prons_percent_above_thres = list(filter(lambda x:x>=thre, prons_percent))
                
                print(f"prons_chs: {prons_chs}, prons_percent: {prons_percent}, above_count: {len(prons_percent_above_thres)}")
                
                if (all(need_fold) and len(prons_percent_above_thres)>=1): # almost all children have the same pron
                    self.charas = charas
                    self.children = defaultdict(PronNode)
                    self.prons = prons
                    print("FOLD: " + ", ".join(prons.keys()))
        print("---- %07d" % id)
    def get(self, loc: Loc) -> 'PronNode':
        if len(self.children) == 0: return self
        for name, node in self.children.items():
            type_ = node.type
            if loc.get(type_) == name:
                return node.get(loc)
    def insert(self, pron: str, chara: str) -> None:
        # if pron not in self.prons:
        #     self.prons[pron] = set()
        self.prons[pron].add(chara)
    def build_node(self, chara: str, loc: Loc, order: List[str]):
        names = loc.get(order[0])
        if len(order) == 1:
            if names not in self.children:
                self.children[names] = PronNode(names, order[0])
                self.children[names].charas = chara
            else:
                self.children[names].charas += chara
        else:
            if names not in self.children:
                self.children[names] = PronNode(names, order[0])
            self.children[names].build_node(chara, loc, order[1:])
    def copy(self) -> 'PronNode':
        node = PronNode(self.names, self.type)
        node.charas = self.charas
        node.prons = self.prons.copy()
        for child in self.children.values():
            node.children[child.names] = child.copy()
        return node
    def isleaf(self) -> bool:
        return len(self.children) == 0
    def delete_empty(self) -> None:
        if self.isleaf(): return
        children = self.children.copy()
        for child in self.children:
            if children[child].isleaf() and len(children[child].prons)==0:
                del children[child]
            else:
                children[child].delete_empty()
                if children[child].isleaf() and len(children[child].prons)==0:
                    del children[child]
        self.children = children
        
    def get_prons(self) -> Dict[str, set]:
        if self.isleaf(): return self.prons
        prons = defaultdict(set)
        for child in self.children.values():
            for pron, pron_chs in child.get_prons().items():
                prons[pron] = prons[pron].union(pron_chs)
        return prons


In [444]:
tree_final = PronNode("", "根")
for chara, locs in chara_to_locate.items():
    if len(locs)!=1: continue
    tree_final.build_node(chara, locs[0], ["攝","韻","呼","等","聲","調"]) #  # ["攝","韻","呼"]
for entry in entry_list:
    if len(entry.multiprons)==1 and len(entry.multiprons[0].prons)==1:
        chara = entry.chara
        pron  = entry.multiprons[0].split_prons[0]
        locs  = chara_to_locate[entry.chara]
        if len(locs) == 1: 
            tree_final.get(locs[0]).insert(get_jpp_final(pron), chara)
tree_final.delete_empty()

In [443]:
tree_final.fold(thre=0.8, fold_empty=True)

FOLDING 0501931: [根]:  { 通江止遇蟹臻山效果假宕梗曾流深咸 } 
FOLDING 0154258: 通[攝]:  { 東冬鍾 } 
FOLDING 0922516: 東[韻]:  { 中 } 
FOLDING 0641275: 中[呼]:  { 一三 } 
FOLDING 0576707: 一[等]:  { 端定溪見明來匣從影清透精並心幫滂 } 
FOLDING 0524399: 端[聲]:  { 平上去 } 
children:  平[調]: ung: 東 , need_fold:  True
children:  上[調]: ung: 董 , need_fold:  True
children:  去[調]: ung: 棟 , need_fold:  True
need_fold: [True, True, True], is_empty: [False, False, False]
prons_chs: 東董棟, prons_percent: [1.0], above_count: 1
FOLD: ung
---- 0524399
children:  端[聲]: ung: 東董棟 , need_fold:  True
FOLDING 0417370: 定[聲]:  { 平上入 } 
children:  平[調]: ung: 桐銅瞳同童 , need_fold:  True
children:  上[調]: ung: 動 , need_fold:  True
children:  入[調]: ung: 讀牘獨犢 , need_fold:  True
need_fold: [True, True, True], is_empty: [False, False, False]
prons_chs: 桐銅瞳同童動讀牘獨犢, prons_percent: [1.0], above_count: 1
FOLD: ung
---- 0417370
children:  定[聲]: ung: 桐銅獨瞳同牘童動犢讀 , need_fold:  True
FOLDING 0767371: 溪[聲]:  { 上入 } 
children:  上[調]: ung: 孔 , need_fold:  True
children:  入[調]: ung: 哭 ,

In [237]:
print(tree_final.str(show_charas=False))

-[根]: 
 -通[攝]: ung: 鳳郁木服紅沐終冢速楓福穆鎔篷農燭牧捅儂寵粟目鍾蜀錄糉絨翁掬宮融捧銃軸觸穀粽鋒鐘東蔥育統聰聾公瞳陸戎逢洪奉讀豐麴欲龍獄勇祿綠蓉工熊牘雄童動叔仲篤貢鹿竹局椶菊弄束蓬甕汞容忠六腫浴肅囑茸促縮痛功冬充蹤甬總棟淑窮屋俗諷隴桐崇銅弓熟蒙拱董躬孔慾毒督鞏沃濃松犢隆腹衝用叢肉卜送夙舂凶築逐宗攏哭膿獨禿辱恭贖庸綜曲癰斛慫通宋蝠族戮同酷擁玉嵩誦, ong: 撲, ang: 疼
 -江[攝]: ong: 嶽龐樁杠學雹岳朔邦扛確巷講蚌腔江項剝缸駁, oeng: 桌卓雙琢, aang: 棒, ung: 捉濁, ang: 握
 -止[攝]: 
  -支[韻]: 
   -開[呼]: 
    -章[聲]: i: 支枝紙肢梔
    -以[聲]: i: 移
    -羣[聲]: ei: 技岐祇
    -曉[聲]: ei: 犧
    -疑[聲]: i: 誼義宜儀議, ai: 蟻
    -常[聲]: i: 豉是
    -日[聲]: i: 爾
    -來[聲]: ei: 璃籬
    -從[聲]: i: 疵
    -精[聲]: i: 紫
    -見[聲]: ei: 羈寄
    -書[聲]: i: 豕翅
    -心[聲]: i: 賜廝斯, ai: 徙璽
    -清[聲]: i: 此雌
    -知[聲]: i: 蜘智知
    -澄[聲]: i: 馳
    -昌[聲]: i: 侈
   -合[呼]: 
    -見[聲]: ai: 詭規
    -曉[聲]: ai: 麾
    -常[聲]: ooi: 垂睡瑞
    -昌[聲]: ooi: 炊
    -邪[聲]: ooi: 隨
    -溪[聲]: ai: 窺虧
    -疑[聲]: ai: 危
    -心[聲]: ooi: 髓
   -[呼]: ei: 碑避皮譬靡俾彼臂婢卑疲脾彌糜
  -脂[韻]: 
   -開[呼]: i: 至肄姊二私尸瓷伊致貳指次姿獅肆矢師自雉資稚嗜咨遲旨夷脂, ei: 利冀棄四痢飢鰭膩肌地姨履死几尼器梨, ai: 篩
   -[呼]: ei: 備丕祕鄙轡媚痹楣秕屁鼻悲庇琵美寐眉
   -合[呼]: 
    -羣[聲]: ai: 櫃葵逵
    -知[聲]: ooi: 追
    -見[聲]: ai: 季癸晷愧軌
    -以[聲]: ai: 惟維
    -來

In [445]:
tree_ZEON = tree_final.children['臻'].children['魂'].copy()
# tree_ZEON = tree_final.children['臻'].copy()

In [453]:
tree_ZEON.fold(thre=0.7)

FOLDING 0529813: 魂[韻]:  { 匣見影明心精從端定清並幫溪曉泥 } 
children:  匣[聲]: an: 混餛棍 , need_fold:  True
children:  見[聲]: an: 骨昆 , need_fold:  True
children:  影[聲]: an: 穩溫 , need_fold:  True
children:  明[聲]: un: 悶沒門 , need_fold:  True
children:  心[聲]: yun: 孫遜損 , need_fold:  True
children:  精[聲]: yun: 尊 , need_fold:  True
children:  從[聲]: yun: 存蹲 , need_fold:  True
children:  端[聲]: an: 墩頓 , need_fold:  True
FOLDING 0158522: 定[聲]:  { 平上去 } 
children:  平[調]: yun: 豚臀, an: 飩 , need_fold:  True
children:  上[調]: yun: 囤 , need_fold:  True
children:  去[調]: an: 鈍 , need_fold:  True
need_fold: [True, True, True], is_empty: [False, False, False]
prons_chs: 豚臀飩囤鈍, prons_percent: [0.75, 0.625], above_count: 1
FOLD: yun, an
---- 0158522
children:  定[聲]: yun: 囤豚臀, an: 飩鈍 , need_fold:  True
children:  清[聲]: yun: 寸村忖, an: 猝 , need_fold:  True
children:  並[聲]: un: 勃脖餑𡋯盆 , need_fold:  True
children:  幫[聲]: un: 本 , need_fold:  True
children:  溪[聲]: an: 困坤窟 , need_fold:  True
children:  曉[聲]: an: 婚昏忽 , need_fold:  True
chi

In [448]:
tree_tmp = tree_ZEON.children['定']#.children['平']

In [489]:
set().union(*tree_tmp.prons.values()).__len__()

5

In [488]:
print(*tree_tmp.prons)

yun an


In [470]:
for loc, node in tree_ZEON.children.items():
    print(node.str(show_charas=False))
    print(list(node.prons.values()))

-匣[聲]: an: 混餛棍
[{'混', '餛', '棍'}]
-見[聲]: an: 骨昆
[{'骨', '昆'}]
-影[聲]: an: 穩溫
[{'穩', '溫'}]
-明[聲]: un: 悶沒門
[{'悶', '沒', '門'}]
-心[聲]: yun: 孫遜損
[{'孫', '遜', '損'}]
-精[聲]: yun: 尊
[{'尊'}]
-從[聲]: yun: 存蹲
[{'存', '蹲'}]
-端[聲]: an: 墩頓
[{'墩', '頓'}]
-定[聲]: yun: 囤豚臀, an: 飩鈍
[{'囤', '豚', '臀'}, {'飩', '鈍'}]
-清[聲]: yun: 寸村忖, an: 猝
[{'寸', '村', '忖'}, {'猝'}]
-並[聲]: un: 勃脖餑𡋯盆
[{'勃', '脖', '餑', '𡋯', '盆'}]
-幫[聲]: un: 本
[{'本'}]
-溪[聲]: an: 困坤窟
[{'困', '坤', '窟'}]
-曉[聲]: an: 婚昏忽
[{'婚', '昏', '忽'}]
-泥[聲]: yun: 嫩
[{'嫩'}]


In [408]:
tree_tmp.prons

defaultdict(set, {'yun': {'臀', '豚'}, 'an': {'飩'}})

In [ ]:
# -平[調]: yun: 豚臀, an: 飩
# -上[調]: yun: 囤
# -去[調]: an: 鈍



In [ ]:
to_be_merged: List[List[str]] = []

In [383]:
[len(node.prons) for node in tree_ZEON.children.values()]

[1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1]